https://googlechromelabs.github.io/chrome-for-testing/
https://github.com/BtbN/FFmpeg-Builds/releases

In [59]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import re

# Настройки Selenium
chrome_options = Options()
chrome_driver_path = "f:/tools/chromedriver.exe"  # Укажите путь к вашему chromedriver

service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

base_url = "https://goliak.com/season-1/episode-1"
driver.get(base_url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "content")))  # ждем, пока загрузится контейнер

# Ожидаем, пока будут загружены все div элементы внутри #content
divs = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content > div"))
)

for el in divs:
            if el.text == 'Я не робот':
                target = el
                break
target.click()

In [60]:
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "iframe"))
)

iframes = driver.find_elements(By.TAG_NAME, "iframe")

for iframe in iframes:
    driver.switch_to.frame(iframe)
    try:
        # Ищем элементы внутри текущего фрейма
        element = driver.find_element(By.ID, "inputData")
        all_videos = (element.get_attribute("textContent"))
    except Exception as e:
        ...
    driver.switch_to.default_content()

In [61]:
series = json.loads(all_videos)

In [62]:
base_url = "https://goliak.com/season-"
episode_suf = "/episode-"

In [63]:
def process_episode(url):
    print(url)
    driver.get(url)
    pattern_ = r"voice=10"
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "iframe"))
    )
    
    videos = driver.find_elements(By.CSS_SELECTOR, '.videos > div.tabss > ul > li')
    
    for video_link in videos:
        if video_link.text == 'Кубик в Кубе':
            video_link.click()
    
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
        
    for iframe in iframes:
        driver.switch_to.frame(iframe)
        try:
            WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CLASS_NAME, "flowplayer"))
            )
            # Ищем элементы внутри текущего фрейма
            m3u8_div = driver.find_element(By.CLASS_NAME, "flowplayer")
            voice = m3u8_div.get_attribute('data-requestfull')
            if re.search(pattern_, voice):        
                print(m3u8 := m3u8_div.get_attribute("data-config"))
                m3u8_link = json.loads(m3u8)
                return m3u8_link['hls']
        except Exception as e:
            ...
        driver.switch_to.default_content()
    


In [ ]:
links = {}
for season in series.keys():
    links[season] = {}
    for episode in series[season].keys():
        if int(season) > 0 and int(episode) > 0:
            print(f"Processing {season}:{episode}")
            links[season][episode] = process_episode(f"{base_url}{season}{episode_suf}{episode}/")

In [65]:
with open('golyak_series.json', "w", encoding = "utf-8") as json_file:
    json.dump(links, json_file, ensure_ascii=False, indent = 4)

In [66]:
links['1']['1']

'https://s5.lomont.site/vod/films/brassic.s01e01.2019.5xrus.web.1080p_419091/hls/index.m3u8'

In [ ]:
ffmpeg -i "https://s5.lomont.site/vod/films/brassic.s01e01.2019.5xrus.web.1080p_419091/hls/index.m3u8" -c copy -bsf:a aac_adtstoasc s1e1.mp4